In [11]:
# script to explore Chicago L stop data for Datascope data challenge:

# data origin: https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Daily-Totals/5neh-572f

# station information origin:
# https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme

# interactive map resource: https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/

import folium
import os
import pandas as pd
from sklearn import preprocessing
print(folium.__version__)
import sys
sys.version

0.2.1


'2.7.12 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:43:17) \n[GCC 4.2.1 (Based on Apple Inc. build 5658) (LLVM build 2336.11.00)]'

In [4]:
# data location (keep raw data off of github):

path_to_data = '/Users/kylefrankovich/Desktop/datascope_CTA_data/' \
               'CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals.csv'

path_to_station_info = '/Users/kylefrankovich/Desktop/datascope_CTA_data/' \
                       'CTA_-_System_Information_-_List_of__L__Stops.csv'


data = pd.read_csv(path_to_data)

station_data = pd.read_csv(path_to_station_info)

print 'size of CTA data:\n', len(data) # 809326 rows, matches data from CTA data site

print '\nsize of station data:\n', len(station_data) # 300 rows, matches data from CTA data site

# let's see what we got:

data.head()

station_data.head()

data.columns # list of column names

station_data.columns

print data.describe()

station_data[station_data['STATION_NAME'] == 'Wilson']['Location']

data.isnull().values.any() # looks like we have no missing data. dope.

station_names = data.stationname.unique()

len(station_names) # 147 station names

station_IDs = data.station_id.unique()

len(station_IDs) # 146 station IDs

map_IDs = station_data.MAP_ID.unique()

len(map_IDs) # 144 map IDs; NB: it appears that map IDs from this dataset
# matches with station IDs in the main dataset, although there is a mismatch in
# number (146 vs. 144); could reflect defunct stations



size of CTA data:
809326

size of station data:
300
          station_id          rides
count  809326.000000  809326.000000
mean    40759.453916    3275.253554
std       444.387019    3144.853450
min     40010.000000       0.000000
25%     40370.000000    1112.000000
50%     40750.000000    2300.000000
75%     41150.000000    4329.000000
max     41690.000000   36323.000000


144

### We need to manipulate our data. What we want is a filtered dataframe containing stop name and number of rides within our selected dates.

In [5]:
#  data mapipulation:

# we're interested in the past 5 years of Lollapalooza data:

lolla_dates = ['07/28/2016', '07/29/2016', '07/30/2016', '07/31/2016',
               '07/31/2015', '08/01/2015', '08/02/2015',
               '08/01/2014', '08/02/2014', '08/03/2014',
               '08/02/2013', '08/03/2013', '08/04/2013',
               '08/03/2012', '08/04/2012', '08/05/2012']

data_filtered_lolla = data[data['date'].isin(lolla_dates)] # data filtered by Lolla dates

print len(data_filtered_lolla)
print data_filtered_lolla.head(10)

# sanity check; let's filter out one station (40010/Austin-Forest Park) to make sure 
# that the ride sum is correct (it checks out):

print '\n Austin-Forest Park data:\n', data_filtered_lolla[data_filtered_lolla['station_id'] == 40010]['rides'].sum()

# collapse across dates by station_id:
#  (NB: here we're collapsing across year, might eventually want to look at year-by-year trends)

data_filtered_lolla = data_filtered_lolla.groupby('station_id').sum()
print data_filtered_lolla.head(10)

print '\n summary of data:\n', data_filtered_lolla.describe()

print '\n sorted, filtered data:\n', data_filtered_lolla.sort_values('rides')

# now need to add column for scaling factor of bubbles:
# example max: 1250; example min: 350


2295
        station_id         stationname        date daytype  rides
600097       40010  Austin-Forest Park  08/03/2012       W   2223
600098       40010  Austin-Forest Park  08/04/2012       A   1176
600099       40010  Austin-Forest Park  08/05/2012       U    926
600128       40020         Harlem-Lake  08/03/2012       W   4678
600129       40020         Harlem-Lake  08/04/2012       A   2790
600130       40020         Harlem-Lake  08/05/2012       U   2394
600159       40030        Pulaski-Lake  08/03/2012       W   2349
600160       40030        Pulaski-Lake  08/04/2012       A   1539
600161       40030        Pulaski-Lake  08/05/2012       U   1231
600190       40040        Quincy/Wells  08/03/2012       W   8146

 Austin-Forest Park data:
23575
             rides
station_id        
40010        23575
40020        55471
40030        23834
40040        69825
40050        52321
40060        68391
40070       145438
40080        81451
40090        32005
40100        55321

 summar

In [6]:
import numpy as np

# testing out plotting with the folium package. I really like it, 
# but here are some alternatives for the type of map I had in mind:

# alternative to get bubble map?
# http://kartograph.org/showcase/symbols/
# I really like the style of this one, perhaps 
# check out the kartograph package later

# or, plotly has some good looking bubbles:
# https://plot.ly/python/bubble-maps/

chicago_coordinates = [41.8781, -87.6298]
m = folium.Map(location=chicago_coordinates, zoom_start=11)
# m.simple_marker(chicago_coordinates)
# folium.CircleMarker(location=chicago_coordinates, radius=1250,
#                     popup='Chicago Coordinates', color='#3186cc',
#                     fill_color='#3186cc').add_to(m)
#folium.Marker(chicago_coordinates, popup='Chicago Coordinates').add_to(m)
division_coordinates = [41.9033, -87.6665]
#folium.Marker(division_coordinates, popup='Division Station').add_to(m)
# folium.CircleMarker(location=division_coordinates, radius=350,
#                     popup='Division Station', color='#3186cc',
#                     fill_color='#3186cc').add_to(m)
wilson_coordinates = [41.964273, -87.657588]
# folium.RegularPolygonMarker(location=wilson_coordinates, popup='Wilson Station',
#                    fill_color='#3186cc', number_of_sides=6, radius=10).add_to(m)

# try adding each station:

for index, row in station_data.iterrows():
    current_ID = row['MAP_ID']
    current_name = row['STOP_NAME']
    string_coords = ''.join(
            c for c in row['Location'] if c not in '(){}<>')
    # coordinates = map(float, string_coords.split(',')) # map was throwing error, try:
    coordinates = [float(i) for i in string_coords.split(',')]
    folium.CircleMarker(location=coordinates, radius=150,
                    popup=current_name, color='#3186cc',
                    fill_color='#3186cc').add_to(m)

m

In [16]:
# let's select the top 25% busiest stops for lolla:

print len(data_filtered_lolla)

n_top_25 = int(len(data_filtered_lolla) * .25)

data_filtered_lolla_subset = data_filtered_lolla.sort_values('rides').iloc[len(data_filtered_lolla)-n_top_25:]
print '\n top 25% data:\n', data_filtered_lolla_subset

# add station name, coordinates:

#first build dict from station_data

station_dict = {}

for index, row in station_data.iterrows():
    current_ID = row['MAP_ID']
    current_name = row['STOP_NAME']
    string_coords = ''.join(
            c for c in row['Location'] if c not in '(){}<>')
    # coordinates = map(float, string_coords.split(',')) # map was throwing error, try:
    coordinates = [float(i) for i in string_coords.split(',')]
    list_to_add = [current_name,coordinates]
    station_dict[current_ID] = list_to_add

print station_dict[40540] # Wilson (Uptown represent)

# print data_filtered_lolla_subset.iloc[0]

station_names = []
station_coords = []
for index, row in data_filtered_lolla_subset.iterrows():
    station_names.append(station_dict[index][0])
    station_coords.append(station_dict[index][1])

# print '\nsubset station names:/n', station_names
# print '\nsubset station coords:/n', station_coords

# add columns from the list
data_filtered_lolla_subset['station_names'] = station_names
data_filtered_lolla_subset['station_coords'] = station_coords

# need to add scaled column manually, which is kind of annoying. 
# I really wish there were examples of folium bubble maps with 
# automatically scaled markers. I'd also like prettier/cleaner 
# markers, perhaps with colorbrewer scaled colors for better
# readability/insight

rides = data_filtered_lolla_subset['rides'].tolist()

min_max_scaler = preprocessing.MinMaxScaler(feature_range=(350,1250))

rides_minmax = min_max_scaler.fit_transform(rides)

rides_scaled = preprocessing.scale(rides)

# X = np.array([[ 1., -1.,  2.],
#               [ 2.,  0.,  0.],
#               [ 0.,  1., -1.]])

# X_scaled = preprocessing.scale(X)

# print '\nscaled X:\n', X_scaled

print '\nrides:\n', rides

print '\nrides scaled:\n', rides_scaled

print '\nrides scaled (min max):\n', rides_minmax

# add scaled ride column
data_filtered_lolla_subset['rides_scaled'] = rides_minmax

print '\n filtered dataset with station names and coordinates:\n', data_filtered_lolla_subset


# might want to filter out loop stations to highlight the neighborhoods, 
# because of course a lot of people are going to the loop on lolla weekends


144

 top 25% data:
             rides
station_id        
40540        73073
40230        73096
41300        74844
40530        79065
40080        81451
40240        81934
40850        82037
40190        82193
41280        84598
40320        86878
40900        87294
40650        88200
41020        91640
40790        93856
40820       101986
41490       104167
40200       111398
40450       114379
40590       116152
40260       125355
40930       125531
40370       132854
40630       133473
40070       145438
41090       147330
41220       166691
40680       169293
40560       177208
41420       182494
40890       188730
41400       198805
40380       201744
41320       202772
40330       223200
41450       246555
41660       265518
['Wilson (Howard-bound)', [41.964273, -87.657588]]

rides:
[73073, 73096, 74844, 79065, 81451, 81934, 82037, 82193, 84598, 86878, 87294, 88200, 91640, 93856, 101986, 104167, 111398, 114379, 116152, 125355, 125531, 132854, 133473, 145438, 147330, 166691, 1692

/Users/kylefrankovich/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/kylefrankovich/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [17]:
# map for top 25% of stations, lolla weekends past 5 years:

m2 = folium.Map(location=chicago_coordinates, zoom_start=11)

for index, row in data_filtered_lolla_subset.iterrows():
#     current_ID = row['station_id']
    current_name = row['station_names']
#     string_coords = ''.join(
#             c for c in row['station_coords'] if c not in '(){}<>')
#     # coordinates = map(float, string_coords.split(',')) # map was throwing error, try:
#     coordinates = [float(i) for i in string_coords.split(',')]
    current_coordinates = row['station_coords']
    current_marker_scale = row['rides_scaled']
    folium.CircleMarker(location=current_coordinates, radius=current_marker_scale,
                    popup=current_name, color='#3186cc',
                    fill_color='#3186cc').add_to(m2)
    
m2

<img src="https://frinkiac.com/gif/S07E09/1304619/1308589.gif?b64lines=IEhleSwgZXZlcnlib2R5ISBJJ20gZ29pbmcKIHRvIGhhdWwgYXNzIHRvCiBMb2xsYXBhbG9vemEh">